In [38]:
import pandas as pd
import numpy as np
import math
import scipy.interpolate
import calendar
import datetime

In [39]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Vehicle Configuration

In [40]:
vehicle_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/KRONE/EXPORT_Ebene_Blockposition_1.csv",engine='python',encoding='latin-1')

In [41]:
vehicle_cols=[#'client',	
#'name 1',	
'equipment',
'designation',
'vehicle type',
'no. Axles',
'vehicle shape',
'axle load type',
'tire type',
#'suspension type',
'Vehicle type/generation',
'superstructure design',
#'Mass in running order',
'axle manufacturer',
'wheel brake type',
'Brake Width / Disc Thickness',
'brake diameter']

In [42]:
config=vehicle_data[vehicle_cols]


# Tyre Changes Dataset

In [43]:
# Reading tyre changes data
tyre_changes_path = "/content/drive/MyDrive/Colab Notebooks/KRONE/tyres.csv"
tyre_changes=pd.read_csv(tyre_changes_path,engine='python',encoding='latin-1')


In [44]:
eng_cols=['fleet',
 'approved',
 'release',
 'service provider',
 'service providerid',
 'service provider country',
 'service provider zip code',
 'contract_number',
 'credit number',
 'service type',
 'contract',
 'user',
 'usernr',
 'Mark',
 'vnr',
 'fin',
 'internaltype',
 'internal group',
 'mileage',
 'zgg',
 'axes',
 'axle load',
 'front axle load',
 'centre axle load',
 'rear axle load',
 'position number',
 'months of age',
 'country of registration',
 'Manufacturer',
 'manufacturer type',
 'first registration',
 'new tires',
 'new width',
 'newcross section',
 'new diameter',
 'new tire size',
 'new profile',
 'noise class_new',
 'noisedb_new',
 'wet adhesion_new',
 'rolling_resistance_new',
 'tyre_class_new',
 'new manufacturer',
 'old tires',
 'old breadth',
 'age cross-section',
 'age diameter',
 'tire size',
 'old profile',
 'noise class_old',
 'noisedb_old',
 'wet adhesion_old',
 'rolling_resistance_old',
 'tire class_old',
 'age maker',
 'remaining tread depth',
 'ground',
 'tire price in eur',
 'Service price in eur']
tyre_changes.columns=eng_cols
tyre_changes.head()

,fleet,approved,release,service provider,service providerid,service provider country,service provider zip code,contract_number,credit number,service type,...,noise class_old,noisedb_old,wet adhesion_old,rolling_resistance_old,tire class_old,age maker,remaining tread depth,ground,tire price in eur,Service price in eur
0,KRONE FLEET A/S,16-10-2013,15-10-2013,"Vergölst VU, Kauffmann, Wilhelmshaven, FB 250",70271,Deutschland,26389,2000141343,E00160707,STANDARD,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,350.00,31.04
1,KRONE FLEET A/S,16-10-2013,15-10-2013,"Vergölst VU, Kauffmann, Wilhelmshaven, FB 250",70271,Deutschland,26389,2000141343,E00160707,STANDARD,...,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,350.00,14.81
2,KRONE FLEET A/S,30-10-2013,02-10-2013,"Vergölst Hamburg Billstr., PS 795",73470,Dänemark,19246,2010140548,E00161881,STANDARD,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,330.46,16.29
3,KRONE FLEET A/S,30-10-2013,02-10-2013,"Vergölst Hamburg Billstr., PS 795",73470,Dänemark,19246,2010140548,E00161881,STANDARD,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,330.46,8.09
4,KRONE FLEET A/S,31-10-2013,28-10-2013,"Vergölst Wittenburg, PS 795 *",74781,Dänemark,19246,2020142426,E00162113,STANDARD,...,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,330.46,8.09


In [45]:
tyre_cols=['release',
 'service type',
 'fin',
'position number',
 'months of age',
 'country of registration',
 'first registration',
 'new tire size',
 'new manufacturer',
 'tire price in eur',
 'Service price in eur']
tyre_changes=tyre_changes[tyre_cols]
tyre_changes.head()

,release,service type,fin,position number,months of age,country of registration,first registration,new tire size,new manufacturer,tire price in eur,Service price in eur
0,15-10-2013,STANDARD,WKESD000000508287,5,24.0,Deutschland,28-09-2011,"385/65 R 22,5",Unknown,350.00,31.04
1,15-10-2013,STANDARD,WKESD000000508287,6,24.0,Deutschland,28-09-2011,"385/65 R 22,5",Unknown,350.00,14.81
2,02-10-2013,STANDARD,WKESD000000407428,1,69.0,Deutschland,02-01-2008,"385/65 R 22,5",Barum,330.46,16.29
3,02-10-2013,STANDARD,WKESD000000407428,2,69.0,Deutschland,02-01-2008,"385/65 R 22,5",Barum,330.46,8.09
4,28-10-2013,STANDARD,WKESD000000407432,1,69.0,Deutschland,02-01-2008,"385/65 R 22,5",Continental,330.46,8.09


In [46]:
tyre_changes['release']=pd.to_datetime(tyre_changes['release'],dayfirst=True)

# Sensor Data

In [47]:
sensor=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KRONE/sensor_wide_final.csv')

In [48]:
try:
  sensor.drop('Unnamed: 0',axis=1,inplace=True)
except:
  pass 

In [49]:
sensor_cols=['device_mongo_id',
 'start_date',
 'BD_EBS_TOTAL_VEHICLE_DISTANCE_max']
sensor_=sensor[sensor_cols]

In [50]:
sensor_=sensor_[sensor_['BD_EBS_TOTAL_VEHICLE_DISTANCE_max']!=0]

# Serial

In [51]:
# Reading the serials data
serial_path = "/content/drive/MyDrive/Colab Notebooks/KRONE/serial.xlsx"
serial = pd.read_excel(serial_path)


In [52]:
serial_=serial[['FIN','MongoID']]
serial_=serial_[serial_['MongoID'].notnull()]
fin_to_remove=['WKESD000000867904','WKESD000000867905','WKESD000000759887']
serial_=serial_[~serial_['FIN'].isin(fin_to_remove)]

# Adding KM to sensor

In [53]:
sensor_=pd.merge(sensor_,serial_,left_on='device_mongo_id',right_on='MongoID')

In [54]:
sensor_.drop('MongoID',inplace=True,axis=1)


In [55]:
sensor_['start_date']=pd.to_datetime(sensor_['start_date'],yearfirst=True)


In [56]:
def km_assign(fin,release):
  temp=sensor_[['start_date','BD_EBS_TOTAL_VEHICLE_DISTANCE_max','FIN']].loc[sensor_['FIN']==fin]
  #print(temp)

  future_date=temp['start_date'][temp['start_date']>=release].min()
  past_date=temp['start_date'][temp['start_date']<=release].max()
  #print(future_date)


  if pd.isnull(future_date):
    future_diff=None
    future_date_km=None    
  else:
    future_date_km=temp[temp['start_date']==future_date]['BD_EBS_TOTAL_VEHICLE_DISTANCE_max'].values[0]
    future_diff=future_date-release
    future_diff=future_diff.days
  
  
  if pd.isnull(past_date):
    past_diff=None 
    past_date_km=None
  else:
    past_date_km=temp[temp['start_date']==past_date]['BD_EBS_TOTAL_VEHICLE_DISTANCE_max'].values[0]
    past_diff=-1*(release-past_date)
    past_diff=past_diff.days 

  return past_diff,past_date_km,future_diff,future_date_km

In [57]:
a=set(tyre_changes['fin'].unique())
b=set(sensor_['FIN'].unique())
common=list(a.intersection(b))

In [58]:
fin_w_sensor=tyre_changes[tyre_changes['fin'].isin(common)]

In [59]:
applied_df = fin_w_sensor.apply(lambda row: km_assign(row.fin,row.release), axis='columns', result_type='expand')
tyre_changes_km = pd.concat([fin_w_sensor, applied_df], axis='columns')


In [60]:
tyre_changes_km = pd.concat([fin_w_sensor, applied_df], axis='columns')

In [61]:
km_cols={0:'past_diff',1:'past_date_km',2:'future_diff',3:'future_date_km'}

In [62]:
tyre_changes_km.rename(columns=km_cols,inplace=True)

In [63]:
tyre_changes_km['nearest_days_km']=tyre_changes_km.apply(lambda row: min((abs(row.past_diff)),row.future_diff),axis='columns')

In [64]:
def approx_km(x1,y1,x2,y2):
  if pd.notnull(x1) and pd.notnull(x2):
    x=[x1,x2]
    y=[y1,y2]
    y_interp = scipy.interpolate.interp1d(x, y)
    return int(y_interp(0))
  elif pd.notnull(x1):
    return y1
  elif pd.notnull(x2):
    return y2
  else:
    return None 

In [65]:
tyre_changes_km['approx_km']=tyre_changes_km.apply(lambda row: 
                                                   approx_km(row.past_diff,row.past_date_km,row.future_diff,row.future_date_km),
                                                   axis='columns')

In [66]:
tyre_changes_km_small=tyre_changes_km[['fin','release','approx_km']].drop_duplicates()
tyre_changes_km_small.reset_index(inplace=True,drop=True)

In [67]:
tyre_changes_km_small['release']=tyre_changes_km_small['release'].apply(lambda row: row.strftime('%d-%m-%Y'))
tyre_changes['release']=tyre_changes['release'].apply(lambda row: row.strftime('%d-%m-%Y'))


In [68]:
tyres_english_km=pd.merge(tyre_changes,tyre_changes_km_small,on=['fin','release'],how='left')

In [69]:
tyres_english_km.head()

,release,service type,fin,position number,months of age,country of registration,first registration,new tire size,new manufacturer,tire price in eur,Service price in eur,approx_km
0,15-10-2013,STANDARD,WKESD000000508287,5,24.0,Deutschland,28-09-2011,"385/65 R 22,5",Unknown,350.00,31.04,NaN
1,15-10-2013,STANDARD,WKESD000000508287,6,24.0,Deutschland,28-09-2011,"385/65 R 22,5",Unknown,350.00,14.81,NaN
2,02-10-2013,STANDARD,WKESD000000407428,1,69.0,Deutschland,02-01-2008,"385/65 R 22,5",Barum,330.46,16.29,NaN
3,02-10-2013,STANDARD,WKESD000000407428,2,69.0,Deutschland,02-01-2008,"385/65 R 22,5",Barum,330.46,8.09,NaN
4,28-10-2013,STANDARD,WKESD000000407432,1,69.0,Deutschland,02-01-2008,"385/65 R 22,5",Continental,330.46,8.09,NaN


# Adding Vehicle config

In [70]:
# Calculate fleet size


In [71]:
tyres_english_km.drop_duplicates(keep='first',inplace=True)

# tyres_english_km['fin_len']=tyres_english_km['fin'].apply(lambda x: len(str(x)))
# tyres_english_km=tyres_english_km[tyres_english_km['fin_len']==17]

def equipment(x):
  try:
    y="F"+str(x)[-7:]
    return y
  except:
    return "Unavailable"  


tyres_english_km['equipment_tyres']=tyres_english_km['fin'].apply(equipment)


In [165]:
tyre_changes_config=pd.merge(tyres_english_km,config,left_on='equipment_tyres',right_on='equipment',how='left')


# Data Preparation

In [166]:
tyre_changes_config.isna().sum()/len(tyre_changes_config)

release                         0.000000
service type                    0.000000
fin                             0.000016
position number                 0.000000
months of age                   0.000465
country of registration         0.194286
first registration              0.000465
new tire size                   0.000000
new manufacturer                0.000000
tire price in eur               0.000000
Service price in eur            0.000016
approx_km                       0.857047
equipment_tyres                 0.000000
equipment                       0.524607
designation                     0.524607
vehicle type                    0.524607
no. Axles                       0.524607
vehicle shape                   0.524607
axle load type                  0.524607
tire type                       0.524607
Vehicle type/generation         0.524607
superstructure design           0.667047
axle manufacturer               0.524607
wheel brake type                0.524607
Brake Width / Di

In [167]:
drop_cols=['equipment','equipment_tyres']
tyre_changes_config.drop(drop_cols,axis=1,inplace=True)

In [168]:
missing_cols=['fin','country of registration','designation',
            'vehicle type','no. Axles', 'vehicle shape','axle load type','tire type','Vehicle type/generation','superstructure design',
            'axle manufacturer','wheel brake type','Brake Width / Disc Thickness','brake diameter']
tyre_changes_config[missing_cols]=tyre_changes_config[missing_cols].fillna('Unknown')

In [169]:
tyre_changes_config=tyre_changes_config[tyre_changes_config['Service price in eur']>=0]

In [170]:
tyre_changes_config=tyre_changes_config[tyre_changes_config['tire price in eur']>0]

In [171]:
tyre_changes_config['Service price in eur'].fillna(tyre_changes_config['Service price in eur'].median(),inplace=True)

In [172]:
tyre_changes_config['Total_Price']=tyre_changes_config['tire price in eur']+tyre_changes_config['Service price in eur']

In [173]:
tyre_changes_config['first registration']=pd.to_datetime(tyre_changes_config['first registration'],dayfirst=True)

In [174]:
# Assumption 
tyre_changes_config['first registration'].fillna(pd.to_datetime("01-10-2013",dayfirst=True),inplace=True)

In [175]:
tyre_changes_config['release']=pd.to_datetime(tyre_changes_config['release'],dayfirst=True)

In [176]:
import datetime
import calendar
def last_date(currentDate):
  y=datetime.date(currentDate.year, currentDate.month, calendar.monthrange(currentDate.year, currentDate.month)[1])
  return y

In [177]:
tyre_changes_config['month_year'] = tyre_changes_config['release'].dt.to_period('M')

In [178]:
tyre_changes_config['as_on_date'] = tyre_changes_config['release'].apply(last_date)

In [179]:
tyre_changes_config['as_on_date']=pd.to_datetime(tyre_changes_config['as_on_date'],yearfirst=True)

In [180]:
#KM cat
def avg_km_pa(a,b):
  if math.isnan(a) | math.isnan(b):
    y='Unknown'
  else:
    a=np.float32(a)
    b=np.float16(b)/12
    try:
      x=a/b
      if x<=5000:
        y= "a.less than 5k"
      elif x<=10000:
        y= "b.5k-10k"
      elif x<=20000:
          y= "c.10k-20k"
      elif x<=30000:
          y= "d.20k-30k"
      elif x<=50000:
          y= "e.30k-50k"
      elif x<=75000:
          y= "f.50k-75k"
      elif x<=100000:
          y= "g.75k-100k"
      elif x<=200000:
          y= "i.100k-200k"
      else:
        y="j.200k & more"
     
    except:
       y="Unknown"        
 
  return y

In [181]:
tyre_changes_config.rename(columns = {'months of age':'months_of_age','first registration':'first_registration'}, inplace = True)

In [182]:
tyre_changes_config['avg_kilometers_per_annum']=tyre_changes_config.apply(lambda x: avg_km_pa(x.approx_km,x.months_of_age), axis='columns')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


In [183]:
#Months of age cat
def age_cat(x):
  if math.isnan(x):
    y="Unknown"    
  else:
    x=np.float16(x)
    if x<=6:
      y= "a.0-6 mon"
    elif x<=12:
      y= "b.6-12 mon"
    elif x<=18:
        y= "c.12-18 mon"
    elif x<=24:
        y= "d.18-24 mon"
    elif x<=30:
        y= "e.24-30 mon"
    elif x<=36:
        y= "f.30-36 mon"
    elif x<=42:
        y= "g.36-42 mon"
    elif x<=48:
        y= "h.42-48 mon"
    elif x<=54:
        y= "i.48-54 mon"
    elif x<=60:
        y= "j.54-60 mon"
    elif x<=96:
        y= "k.5-8 yrs"
    elif x<=144:
        y= "l.8-12 yrs"
    elif x<=180:
        y= "m.12-15 yrs"
    else:
      y="n.15+ yrs"   
         
 
  return y

In [184]:
tyre_changes_config['Vehicle Age']=tyre_changes_config['months_of_age'].apply(age_cat)

In [185]:
#Tire changes
tyre_changes_config['no.of.tyre_changes']=1

In [186]:
tyre_changes_config.to_csv('/content/drive/MyDrive/Colab Notebooks/KRONE/changes_config_km.csv',index=False)

In [187]:
import pandas as pd

In [188]:
#tyre_changes_config=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KRONE/changes_config_km.csv')

In [189]:
tyre_changes_config.dtypes

release                         datetime64[ns]
service type                            object
fin                                     object
position number                          int64
months_of_age                          float64
country of registration                 object
first_registration              datetime64[ns]
new tire size                           object
new manufacturer                        object
tire price in eur                      float64
Service price in eur                   float64
approx_km                              float64
designation                             object
vehicle type                            object
no. Axles                               object
vehicle shape                           object
axle load type                          object
tire type                               object
Vehicle type/generation                 object
superstructure design                   object
axle manufacturer                       object
wheel brake t

In [190]:
list(tyre_changes_config)

['release',
 'service type',
 'fin',
 'position number',
 'months_of_age',
 'country of registration',
 'first_registration',
 'new tire size',
 'new manufacturer',
 'tire price in eur',
 'Service price in eur',
 'approx_km',
 'designation',
 'vehicle type',
 'no. Axles',
 'vehicle shape',
 'axle load type',
 'tire type',
 'Vehicle type/generation',
 'superstructure design',
 'axle manufacturer',
 'wheel brake type',
 'Brake Width / Disc Thickness',
 'brake diameter',
 'Total_Price',
 'month_year',
 'as_on_date',
 'avg_kilometers_per_annum',
 'Vehicle Age',
 'no.of.tyre_changes']

In [191]:
vehicle_cols=[
 'fin',
 'country of registration',
 'first_registration',
 'designation',
 'vehicle type',
 'no. Axles',
 'vehicle shape',
 'axle load type',
 'tire type',
 'Vehicle type/generation',
 'superstructure design',
 'axle manufacturer',
 'wheel brake type',
 'Brake Width / Disc Thickness',
 'brake diameter']
vehicle_df=tyre_changes_config[vehicle_cols]
vehicle_df.drop_duplicates(inplace=True)
vehicle_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,fin,country of registration,first_registration,designation,vehicle type,no. Axles,vehicle shape,axle load type,tire type,Vehicle type/generation,superstructure design,axle manufacturer,wheel brake type,Brake Width / Disc Thickness,brake diameter
0,WKESD000000508287,Deutschland,2011-09-28,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
2,WKESD000000407428,Deutschland,2008-01-02,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
4,WKESD000000407432,Deutschland,2008-01-02,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
6,WKESD000000407427,Deutschland,2008-02-11,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
10,WKESD000000407418,Deutschland,2008-02-20,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61323,WKESZ000000453641,Deutschland,2009-05-26,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
61340,WKESD000000407117,Deutschland,2007-11-27,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
61392,WKESZ000000434055,Deutschland,2008-08-21,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
61595,WKESD000000592756,Deutschland,2013-10-10,SDC 27 eLTU4 Box Liner,S Sattel,D dreiachs-,C Container,27 t,e einzelbereift,"TU4 Tieflader, gekröpft",Unknown,BPW,Scheibe,Unknown,"430, Scheibendurchm. 22.5 Zoll"


In [192]:
#monthly fleet
fleet=vehicle_df[['first_registration','fin']]
fleet.drop_duplicates(inplace=True)
fleet.reset_index(inplace=True,drop=True)
#fleet['as_on_date'] = fleet['first_registration'].apply(last_date)
#fleet['month_year'] = fleet['first_registration'].dt.to_period('M')
#fleet.drop('first_registration',inplace=True,axis=1)
fleet
# fleet_=pd.DataFrame(fleet.groupby('month_year')['fin'].reset_index())
# fleet_

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,first_registration,fin
0,2011-09-28,WKESD000000508287
1,2008-01-02,WKESD000000407428
2,2008-01-02,WKESD000000407432
3,2008-02-11,WKESD000000407427
4,2008-02-20,WKESD000000407418
...,...,...
9565,2009-05-26,WKESZ000000453641
9566,2007-11-27,WKESD000000407117
9567,2008-08-21,WKESZ000000434055
9568,2013-10-10,WKESD000000592756


In [193]:
idx = pd.date_range(tyre_changes_config['as_on_date'].min(),tyre_changes_config['as_on_date'].max(),freq='M')
idx
# fleet.index = pd.DatetimeIndex(fleet.index.to_timestamp(freq='M'))
# fleet_ = fleet.reindex(idx)
# fleet_.reset_index(inplace=True)
# fleet_['month_year']=fleet_['index'].dt.to_period('M')
# fleet_

DatetimeIndex(['2013-10-31', '2013-11-30', '2013-12-31', '2014-01-31',
               '2014-02-28', '2014-03-31', '2014-04-30', '2014-05-31',
               '2014-06-30', '2014-07-31',
               ...
               '2021-10-31', '2021-11-30', '2021-12-31', '2022-01-31',
               '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31',
               '2022-06-30', '2022-07-31'],
              dtype='datetime64[ns]', length=106, freq='M')

In [194]:
fleet_all = pd.MultiIndex.from_product([fleet['fin'].unique(), idx]).to_frame().reset_index(drop=True)
fleet_all.columns = ['fin', 'as_on_date']
fleet_all

,fin,as_on_date
0,WKESD000000508287,2013-10-31
1,WKESD000000508287,2013-11-30
2,WKESD000000508287,2013-12-31
3,WKESD000000508287,2014-01-31
4,WKESD000000508287,2014-02-28
...,...,...
1014415,WKESD000000418184,2022-03-31
1014416,WKESD000000418184,2022-04-30
1014417,WKESD000000418184,2022-05-31
1014418,WKESD000000418184,2022-06-30


In [195]:
fleet_base=pd.merge(fleet_all,fleet,on='fin',how='left')
fleet_base['temp']=(fleet_base['as_on_date']-fleet_base['first_registration'])/np.timedelta64(1, 'D')
fleet_base['active_fleet']=fleet_base['temp'].apply(lambda x: 1 if x>=0 else 0)
fleet_base=fleet_base[fleet_base['active_fleet']==1]
fleet_base.drop('temp',axis=1,inplace=True)
fleet_base

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,fin,as_on_date,first_registration,active_fleet
0,WKESD000000508287,2013-10-31,2011-09-28,1
1,WKESD000000508287,2013-11-30,2011-09-28,1
2,WKESD000000508287,2013-12-31,2011-09-28,1
3,WKESD000000508287,2014-01-31,2011-09-28,1
4,WKESD000000508287,2014-02-28,2011-09-28,1
...,...,...,...,...
1014415,WKESD000000418184,2022-03-31,2008-03-11,1
1014416,WKESD000000418184,2022-04-30,2008-03-11,1
1014417,WKESD000000418184,2022-05-31,2008-03-11,1
1014418,WKESD000000418184,2022-06-30,2008-03-11,1


In [199]:
fleet_base.duplicated().sum()

0

In [196]:
changes_cols=['release',
 'service type',
 'fin',
 'position number',
 'months_of_age',
 'new tire size',
 'new manufacturer',
 'tire price in eur',
 'Service price in eur',
 'approx_km',
 'Total_Price',
 'month_year',
 'as_on_date',
 'avg_kilometers_per_annum',
 'Vehicle Age',
 'no.of.tyre_changes']

tran_df=tyre_changes_config[changes_cols]



In [197]:
tran_df.duplicated().sum()

0

In [198]:
tran_df.drop_duplicates(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [204]:
tran_df.to_csv('/content/drive/MyDrive/Colab Notebooks/KRONE/transactions_data.csv',index=False)
vehicle_df.to_csv('/content/drive/MyDrive/Colab Notebooks/KRONE/vehicles_data.csv',index=False)
fleet_base.to_csv('/content/drive/MyDrive/Colab Notebooks/KRONE/fleet_monthly_base.csv',index=False)

In [201]:
vehicle_df.shape

(10902, 15)

In [202]:
tran_df.shape

(58655, 16)

In [203]:
fleet_base.shape

(750322, 4)

In [208]:
fleet_base.head(30)

,fin,as_on_date,first_registration,active_fleet
0,WKESD000000508287,2013-10-31,2011-09-28,1
1,WKESD000000508287,2013-11-30,2011-09-28,1
2,WKESD000000508287,2013-12-31,2011-09-28,1
3,WKESD000000508287,2014-01-31,2011-09-28,1
4,WKESD000000508287,2014-02-28,2011-09-28,1
5,WKESD000000508287,2014-03-31,2011-09-28,1
6,WKESD000000508287,2014-04-30,2011-09-28,1
7,WKESD000000508287,2014-05-31,2011-09-28,1
8,WKESD000000508287,2014-06-30,2011-09-28,1
9,WKESD000000508287,2014-07-31,2011-09-28,1


In [206]:
tran_df.head()

,release,service type,fin,position number,months_of_age,new tire size,new manufacturer,tire price in eur,Service price in eur,approx_km,Total_Price,month_year,as_on_date,avg_kilometers_per_annum,Vehicle Age,no.of.tyre_changes
0,2013-10-15,STANDARD,WKESD000000508287,5,24.0,"385/65 R 22,5",Unknown,350.00,31.04,NaN,381.04,2013-10,2013-10-31,Unknown,d.18-24 mon,1
1,2013-10-15,STANDARD,WKESD000000508287,6,24.0,"385/65 R 22,5",Unknown,350.00,14.81,NaN,364.81,2013-10,2013-10-31,Unknown,d.18-24 mon,1
2,2013-10-02,STANDARD,WKESD000000407428,1,69.0,"385/65 R 22,5",Barum,330.46,16.29,NaN,346.75,2013-10,2013-10-31,Unknown,k.5-8 yrs,1
3,2013-10-02,STANDARD,WKESD000000407428,2,69.0,"385/65 R 22,5",Barum,330.46,8.09,NaN,338.55,2013-10,2013-10-31,Unknown,k.5-8 yrs,1
4,2013-10-28,STANDARD,WKESD000000407432,1,69.0,"385/65 R 22,5",Continental,330.46,8.09,NaN,338.55,2013-10,2013-10-31,Unknown,k.5-8 yrs,1


In [207]:
vehicle_df.head()

,fin,country of registration,first_registration,designation,vehicle type,no. Axles,vehicle shape,axle load type,tire type,Vehicle type/generation,superstructure design,axle manufacturer,wheel brake type,Brake Width / Disc Thickness,brake diameter
0,WKESD000000508287,Deutschland,2011-09-28,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
2,WKESD000000407428,Deutschland,2008-01-02,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
4,WKESD000000407432,Deutschland,2008-01-02,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
6,WKESD000000407427,Deutschland,2008-02-11,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
10,WKESD000000407418,Deutschland,2008-02-20,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


In [7]:
# group_cols=['service type','fin','position number','country of registration',
#             'new tire size','new manufacturer','designation','vehicle type',
#             'no. Axles','vehicle shape','axle load type','tire type',
#             'Vehicle type/generation','superstructure design','axle manufacturer',
#             'wheel brake type','Brake Width / Disc Thickness','brake diameter',
#             'month_year','as_on_date','avg_kilometers_per_annum','Vehicle Age']
# measure_cols=['tire price in eur','Service price in eur','Total_Price','no.of.tyre_changes']


In [13]:
# tyre_changes_config[group_cols].nunique().prod()

5167805780716978176

In [15]:
# import numpy as np

In [16]:
# tyre_changes_config_int=tyre_changes_config
# tyre_changes_config_int[group_cols]=tyre_changes_config_int[group_cols].astype('category')
# tyre_changes_config_int[measure_cols]=tyre_changes_config_int[measure_cols].astype(np.float16)

In [17]:
# df_dash=tyre_changes_config_int.groupby(group_cols)[measure_cols].sum().reset_index()
# df_dash.head()

ValueError: ignored

In [ ]:
# df_dash=pd.merge(df_dash,fleet_,on='month_year',how='left')

In [ ]:
# df_dash.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58620 entries, 0 to 58619
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   service type                  58620 non-null  object 
 1   fin                           58620 non-null  object 
 2   position number               58620 non-null  object 
 3   country of registration       58620 non-null  object 
 4   new tire size                 58620 non-null  object 
 5   new manufacturer              58620 non-null  object 
 6   designation                   58620 non-null  object 
 7   vehicle type                  58620 non-null  object 
 8   no. Axles                     58620 non-null  object 
 9   vehicle shape                 58620 non-null  object 
 10  axle load type                58620 non-null  object 
 11  tire type                     58620 non-null  object 
 12  Vehicle type/generation       58620 non-null  object 
 13  s

In [ ]:
# df_dash.to_csv('/content/drive/MyDrive/Colab Notebooks/KRONE/final_dashboard_data.csv',index=False)